In [1]:
from transformers import pipeline

In [4]:
classifier = pipeline("fill-mask")
result = classifier("This is a <mask> example.")
print(result)

No model was supplied, defaulted to distilbert/distilroberta-base and revision fb53ab8 (https://huggingface.co/distilbert/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at distilbert/distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


[{'score': 0.07740752398967743, 'token': 205, 'token_str': ' good', 'sequence': 'This is a good example.'}, {'score': 0.049008261412382126, 'token': 6369, 'token_str': ' concrete', 'sequence': 'This is a concrete example.'}, {'score': 0.04729151725769043, 'token': 6097, 'token_str': ' typical', 'sequence': 'This is a typical example.'}, {'score': 0.041539888828992844, 'token': 1969, 'token_str': ' perfect', 'sequence': 'This is a perfect example.'}, {'score': 0.03892398625612259, 'token': 31046, 'token_str': ' textbook', 'sequence': 'This is a textbook example.'}]


d:\Softwares\Anaconda\anaconda\envs\ai\lib\site-packages\transformers\models\roberta\modeling_roberta.py:370: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [8]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
outputs = model(**inputs)
print(outputs.logits.shape)
print("--------------")
print(outputs.logits)
print("--------------")
import torch.nn.functional as F
probs = F.softmax(outputs.logits, dim=-1)
print(probs)

torch.Size([2, 2])
--------------
tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>)
--------------
tensor([[4.0195e-02, 9.5980e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward0>)


In [9]:
from torch import nn
from transformers import AutoConfig
from transformers import AutoTokenizer

In [13]:
model_ckpt = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

text = "time flies like an arrow"
inputs = tokenizer(text, return_tensors="pt")
print(inputs.input_ids.shape)
print("==============")
print(inputs.input_ids)

torch.Size([1, 7])
tensor([[  101,  2051, 10029,  2066,  2019,  8612,   102]])


In [16]:
config = AutoConfig.from_pretrained(model_ckpt)
token_emb = nn.Embedding(config.vocab_size, config.hidden_size)
print(token_emb)
print("==============")
input_emb = token_emb(inputs.input_ids)
print(input_emb.size())

Embedding(30522, 768)
torch.Size([1, 7, 768])


In [24]:
import torch
from math import sqrt

Q = K = V = input_emb
dim_k = K.size(-1)
# print(dim_k)
scores = torch.bmm(Q, K.transpose(-2, -1)) / sqrt(dim_k)
print(scores.size())
print("==============")
# print(scores)
attn_weights = torch.nn.functional.softmax(scores, dim=-1)
print(attn_weights.size())
print("============")
print(attn_weights.sum(dim=-1))

torch.Size([1, 7, 7])
torch.Size([1, 7, 7])
tensor([[1., 1., 1., 1., 1., 1., 1.]], grad_fn=<SumBackward1>)
